In [1]:
import pandas as pd
import os
import glob
from datetime import datetime

In [2]:
def transform_menu_data(folder_path):
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    
    all_data = []
    
    for file_path in excel_files:
        df = pd.read_excel(file_path, sheet_name='menu')
        meal_types = {
            '조식': 'Breakfast',
            '중식': 'Lunch',
            '석식': 'Dinner'
        }
        
        date_columns = [col for col in df.columns if str(col).startswith('2024')]
        
        for date_col in date_columns:
            date_str = str(date_col).split('00:00:00')[0].strip()
            day = pd.to_datetime(date_str).strftime('%Y-%m-%d')
            
            for meal_kor, meal_eng in meal_types.items():
                meal_rows = df[df['구분'] == meal_kor]
                
                if not meal_rows.empty:
                    menus = meal_rows[date_col].tolist()
                    menus = [menu for menu in menus if pd.notna(menu)]
                    menu_str = ', '.join(menus)
                    all_data.append({
                        'Day': day,
                        'MealType': meal_eng,
                        'Menus': menu_str
                    })
    
    result_df = pd.DataFrame(all_data)
    result_df = result_df.sort_values(['Day', 'MealType'])

    result_df.to_excel(f'../processed_DB/DIET_{folder_path}.xlsx', index=False)
    return result_df

In [3]:
def combine_menus(folder_path):
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    
    combined_nutrient = pd.DataFrame()
    combined_category = pd.DataFrame()
    combined_ingredient = pd.DataFrame()
    
    for file_path in excel_files:
        try:
            nutrient_df = pd.read_excel(file_path, sheet_name='nutrient')
            combined_nutrient = pd.concat([combined_nutrient, nutrient_df], ignore_index=True)
            
            category_df = pd.read_excel(file_path, sheet_name='category')
            combined_category = pd.concat([combined_category, category_df], ignore_index=True)

            ingredient_df = pd.read_excel(file_path, sheet_name='ingredient')
            ingredient_df = ingredient_df[['Menu', 'Ingredient', 'Amount_g']]
            combined_ingredient = pd.concat([combined_ingredient, ingredient_df], ignore_index=True)
                       
        except Exception as e:
            print(f"Error processing {os.path.basename(file_path)}: {str(e)}")
    
    combined_nutrient = combined_nutrient.drop_duplicates()
    combined_category = combined_category.drop_duplicates()
    combined_ingredient = combined_ingredient.drop_duplicates()
    
    with pd.ExcelWriter(f'../processed_DB/Menu_ingredient_nutrient_{folder_path}.xlsx') as writer:
        combined_nutrient.to_excel(writer, sheet_name='nutrient', index=False)
        combined_category.to_excel(writer, sheet_name='category', index=False)
        combined_ingredient.to_excel(writer, sheet_name='ingredient', index=False)
    
    return combined_nutrient, combined_category, combined_ingredient

In [4]:
def combine_price(folder_path):
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    combined_price = pd.DataFrame()
    
    for file_path in excel_files:
        try:
            ingredient_df = pd.read_excel(file_path, sheet_name='ingredient')
            ingredient_df = ingredient_df[['Ingredient', '단가(원/g)']]
            combined_price = pd.concat([combined_price, ingredient_df], ignore_index=True)
                       
        except Exception as e:
            print(f"Error processing {os.path.basename(file_path)}: {str(e)}")
    
    combined_price = combined_price.drop_duplicates()
    
    with pd.ExcelWriter(f'../processed_DB/Ingredient_Price_{folder_path}.xlsx') as writer:
        combined_price.to_excel(writer, sheet_name='Sheet1', index=False)
    
    return combined_price

In [5]:
# 사용 예시
folder_path = 'sarang'
result_df = transform_menu_data(folder_path)
nutrient_df, category_df, ingredient_df = combine_menus(folder_path)
combined_price = combine_price(folder_path)